In [2]:
import numpy as np
from sklearn.preprocessing import StandardScaler

In [3]:
X = np.load('C:\\Users\\47575909\\Desktop\\X_1.npy')

In [4]:
#aplano los datos para que la forma sea (3044, 14*230*384) para que cada trial de cada eeg sea un vector unidimensional
X = X.reshape(3044, -1)

In [5]:
X.shape

(3044, 1236480)

In [ ]:
from sklearn.decomposition import PCA

variance_to_keep = 0.95  #me quedo con el 95% de la varianza, esto es flexible (mientras más porcentaje, mayor va  quedar la dimensionalidad de mi X)

pca = PCA()
X = pca.fit_transform(X)  #aplico pca para los datos escalados

#Encuentro el número mínimo de componentes principales para alcanzar la varianza deseada
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
#('n_components_to_keep' contendrá el número mínimo de componentes principales necesarios para explicar al menos el 95% de la varianza en los datos.)
n_components_to_keep = np.argmax(cumulative_variance_ratio >= variance_to_keep) + 1 #chona me dijo que me deberían quedar por ejemplo 1000 atributos

#Selecciono la cant de componentes principales que me salió
X = X[:, :n_components_to_keep]

print("me quedé con " + n_components_to_keep + "componentes principales")

In [ ]:
X.shape

In [ ]:
print("Varianza explicada por cada componente principal:", pca.explained_variance_ratio_)

In [ ]:
#no se si lo que hice está bien, chona me dijo que usara PaCMAP pero no lo entiendo, y acá encontre que con sklearn medio que se puede hacer asi que :)